# Abstract

In [54]:
#importing libraries
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [55]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1                        # Number of parallel threads used to run Algorithms
min_mem_size=6 
classification= True
scale=False
max_models=None    
model_path=None
balance_y=False 
balance_threshold=0.2             
name=None 
server_path=None  
analysis=0 

When enabled, H2O will either undersample the majority classes or oversample the minority classes. Note that the resulting model will also correct the final probabilities (“undo the sampling”) using a monotonic transform, so the predicted probabilities of the first model will differ from a second model. However, because AUC only cares about ordering, it won’t be affected.

In [56]:
data_path='C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv'

### Data Cleaning

In [57]:
data=pd.read_csv("Employee_access_data.csv", decimal = ',')

In [58]:
data.dtypes

ACTION              int64
RESOURCE            int64
MGR_ID              int64
ROLE_ROLLUP_1       int64
ROLE_ROLLUP_2       int64
ROLE_DEPTNAME       int64
ROLE_TITLE          int64
ROLE_FAMILY_DESC    int64
ROLE_FAMILY         int64
ROLE_CODE           int64
dtype: object

**Checking for the Null values**

In [59]:
total = data.isnull().sum()[data.isnull().sum() != 0].sort_values(ascending = False)
percent = pd.Series(round(total/len(data)*100,2))
pd.concat([total, percent], axis=1, keys=['total_missing', 'percent'])

,total_missing,percent


No null values

### RUNTIME

In [60]:
run_time=500

In [61]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size, "GB")

2 GB


In [62]:
#Defining functions

#generating random run_id
def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str

# storing in m_data dictionary  
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['target']=target
  m_data['classification']=classification
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['max_models']=model
#   m_data['Shape_Rows*Cols'] = shape
  m_data['start_time'] = time.time()
#   m_data['model_execution_time'] = execution
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model 
  m_data['scale']=scale  
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()  
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data

#converting dictionary to json
def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
# Segregating in different lists of int, enum, reals and checking for missing values and then scaling(standardizing)    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return

# Determining Independent variables (X) from the dataset 
def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x

#Checkinh if X does not exist then remove
def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    

### RUN_ID

In [63]:
#randomly generating run_id through alphabet function
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id) 

y2l466LD5


In [64]:
# # Creating a log file
# logfile=run_id+'_autoh2o_log.zip'
# logs_path=os.path.join(run_dir,'logs')
# print(logs_path,' ',logfile)

In [65]:
# 65535 Highest port no
port_no=random.randint(5555,55555)

#initializing H2O
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:20617 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.2+9-LTS, mixed mode)
  Starting server from C:\Users\hp\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\hp\AppData\Local\Temp\tmpk_sgf0fg
  JVM stdout: C:\Users\hp\AppData\Local\Temp\tmpk_sgf0fg\h2o_hp_started_from_python.out
  JVM stderr: C:\Users\hp\AppData\Local\Temp\tmpk_sgf0fg\h2o_hp_started_from_python.err
  Server is running at http://127.0.0.1:20617
Connecting to H2O server at http://127.0.0.1:20617 ... successful.


H2O cluster uptime:,07 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_hp_3sxpog
H2O cluster total nodes:,1
H2O cluster free memory:,2 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [66]:
print(data_path)              #path of csv 

C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv


In [67]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,test_path,model_path,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)  

{'target': None, 'classification': True, 'run_time': 500, 'run_id': 'y2l466LD5', 'max_models': None, 'start_time': 1555387074.0489674, 'server_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project', 'data_path': 'C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv', 'test_path': None, 'scale': False, 'model_path': None, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1555387074.0489674, 'run_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project\\y2l466LD5', 'nthreads': 1, 'min_mem_size': 2, 'analysis': 0}


In [68]:
#importing data file on h2o server
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [69]:
df.head()

ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,17183,1540,117961,118343,123125,118536,118536,308574,118539
1,36724,14457,118219,118220,117884,117879,267952,19721,117880
1,36135,5396,117961,118343,119993,118321,240983,290919,118322
1,42680,5905,117929,117930,119569,119323,123932,19793,119325
0,45333,14561,117951,117952,118008,118568,118568,19721,118570
1,25993,17227,117961,118343,123476,118980,301534,118295,118982
1,19666,4209,117961,117969,118910,126820,269034,118638,126822
1,31246,783,117961,118413,120584,128230,302830,4673,128231
1,78766,56683,118079,118080,117878,117879,304519,19721,117880


In [70]:
rows = len(df)
print("Total rows in the data set = ", rows)
cols = len(df.columns)
print("Total Columns in the data set = ", cols)

shape = df.shape
print("rows X columns = ", shape)

meta_data['Shape_Rows*Cols'] = shape

Total rows in the data set =  32769
Total Columns in the data set =  10
rows X columns =  (32769, 10)


In [71]:
df.describe()

Rows:32769
Cols:10




,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
type,int,int,int,int,int,int,int,int,int,int
mins,0.0,0.0,25.0,4292.0,23779.0,4674.0,117879.0,4673.0,3130.0,117880.0
mean,0.9421099209618847,42923.916170771125,25988.957978577324,116952.62778845867,118301.82315603165,118912.77991394309,125916.15264426745,170178.36964814307,183703.40889255097,119789.43013213709
maxs,1.0,312153.0,311696.0,311178.0,286791.0,286792.0,311867.0,311867.0,308574.0,270691.0
sigma,0.23353903780676308,34173.892702138255,35928.03165014073,10875.563591093745,4551.588572012568,18961.32291708769,31036.465824743256,69509.46213013002,100488.40741337684,5784.275515531029
zeros,1897,13,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0
0,1.0,39353.0,85475.0,117961.0,118300.0,123472.0,117905.0,117906.0,290919.0,117908.0
1,1.0,17183.0,1540.0,117961.0,118343.0,123125.0,118536.0,118536.0,308574.0,118539.0
2,1.0,36724.0,14457.0,118219.0,118220.0,117884.0,117879.0,267952.0,19721.0,117880.0


In [72]:
'''
import sys
sys.stdout = open('describe.txt', 'w')
print ('test')
'''

"\nimport sys\nsys.stdout = open('describe.txt', 'w')\nprint ('test')\n"

### Depentdent Variable

In [73]:
# dependent variable
# assign target an d inputs for classification or regression
if target==None:
  target="ACTION"   
y = target
y

'ACTION'

### Independent Variables

In [74]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [75]:
# independent variables
# putting all independent variables in the list X 

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
  print(X)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables to meta data

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']


### Problem Type

The dependent variable for this data set is of Binary/ classification type.

In [76]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [77]:
df[y]

ACTION
1
1
1
1
1
0
1
1
1
1


**Checking the number of classes of dependent variable**

In [78]:
# Total categories in the target column
lvl = df[y].levels()
print(lvl)
meta_data["levels"] = lvl

[['0', '1']]


y has 2 levels or classes

In [79]:
# checking value of target for real, int, enum
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok   

In [80]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [81]:
# since y is enum type
print(ok)

True


In [82]:
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    

In [83]:
# getting the data types of all the variables in the dataset
allV=get_variables_types(df)
allV

{'ACTION': 'enum',
 'RESOURCE': 'int',
 'MGR_ID': 'int',
 'ROLE_ROLLUP_1': 'int',
 'ROLE_ROLLUP_2': 'int',
 'ROLE_DEPTNAME': 'int',
 'ROLE_TITLE': 'int',
 'ROLE_FAMILY_DESC': 'int',
 'ROLE_FAMILY': 'int',
 'ROLE_CODE': 'int'}

In [84]:
# Adding the data types to meta data 
meta_data['variables']=allV

### Using H2OAutoML

* Used for automatic training and tuning of the models

In [85]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [86]:
model_start_time = time.time()

In [87]:
aml.train(x=X,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [88]:
execution_time =  time.time() - model_start_time
meta_data['model_execution_time_sec'] = execution_time
print("Execution time for ", run_time,"sec =  ",meta_data['model_execution_time_sec'])

Execution time for  500 sec =   528.6322100162506


## LeaderBoard

In [89]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [90]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,StackedEnsemble_BestOfFamily_AutoML_20190415_2...,0.861552,0.156262,0.335448,0.200608,0.040244
1,StackedEnsemble_AllModels_AutoML_20190415_235801,0.860448,0.156327,0.329694,0.200633,0.040254
2,GBM_grid_1_AutoML_20190415_235801_model_1,0.860402,0.170265,0.336112,0.212300,0.045071
3,XRT_1_AutoML_20190415_235801,0.848309,0.170001,0.360640,0.202310,0.040929
4,DRF_1_AutoML_20190415_235801,0.847216,0.176933,0.350980,0.201532,0.040615
5,GBM_4_AutoML_20190415_235801,0.843034,0.159992,0.383300,0.203426,0.041382
6,GBM_3_AutoML_20190415_235801,0.839973,0.163117,0.390055,0.206017,0.042443
7,GBM_5_AutoML_20190415_235801,0.836560,0.164695,0.410981,0.206961,0.042833
8,GBM_2_AutoML_20190415_235801,0.834621,0.165739,0.420936,0.207791,0.043177
9,GBM_1_AutoML_20190415_235801,0.825490,0.169833,0.424946,0.210397,0.044267


In [91]:
length = len(aml_leaderboard_df)- 1
length
meta_data["Models_generated"] = length

### Saving Leaderboard into csv

In [92]:
# save leaderboard to csv
# run_time = run_time.ascharacter()
leaderboard_stats=run_id+'EmpAccess_500' + '_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

### Generating hyperparameters for the best model on the leaderboard

In [93]:
# STart best model as first model
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [94]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'StackedEnsemble_BestOfFamily_AutoML_20190415_235801',
   'type': 'Key<Model>',
   'URL': '/3/Models/StackedEnsemble_BestOfFamily_AutoML_20190415_235801'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_py_13_sid_870a',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_py_13_sid_870a'}},
 'response_column': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ColSpecifierV3',
    'schema_type': 'VecSpecifier'},
   'column_name': 'ACTION',
   'is_member_of_frames': None}},
 'validation_frame': {'default': None, 'actual': None},
 'blending_frame': {'default': None, 'actual': None},
 'base_models': {'default': [],
  'actual': [{'__meta': {'schema_version': 3,
     'sc

In [95]:
# generating normalized coeff.
mods=mod_best.coef_norm
print(mods)


Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190415_235801
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.02011199327953417
RMSE: 0.141816759515701
LogLoss: 0.08107665014737735
Null degrees of freedom: 32768
Residual degrees of freedom: 32764
Null deviance: 14491.899767390338
Residual deviance: 5313.601497358816
AIC: 5323.601497358816
AUC: 0.9922240528443118
pr_auc: 0.9956375728848502
Gini: 0.9844481056886236
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7513047464307223: 


,0,1,Error,Rate
0,1477.0,420.0,0.2214,(420.0/1897.0)
1,316.0,30556.0,0.0102,(316.0/30872.0)
Total,1793.0,30976.0,0.0225,(736.0/32769.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.7513047,0.9880999,176.0
max f2,0.2649637,0.9913921,313.0
max f0point5,0.8862056,0.9913736,122.0
max accuracy,0.7513047,0.9775397,176.0
max precision,0.9828978,1.0,0.0
max recall,0.0291408,1.0,386.0
max specificity,0.9828978,1.0,0.0
max absolute_mcc,0.8497435,0.7994737,138.0
max min_per_class_accuracy,0.9237428,0.9634296,99.0
max mean_per_class_accuracy,0.9393746,0.9654607,86.0


Gains/Lift Table: Avg response rate: 94.21 %, avg score: 93.46 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.6067619,0.9743942,240.0
max f2,0.0430560,0.9880303,386.0
max f0point5,0.8549880,0.9706867,150.0
max accuracy,0.6527983,0.9511123,228.0
max precision,0.9830328,0.9950739,1.0
max recall,0.0060613,1.0,398.0
max specificity,0.9838748,0.9994729,0.0
max absolute_mcc,0.8549880,0.4920161,150.0
max min_per_class_accuracy,0.9650919,0.7904898,54.0
max mean_per_class_accuracy,0.9548999,0.7990502,72.0


Gains/Lift Table: Avg response rate: 94.21 %, avg score: 94.21 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100095,0.9824838,1.0517389,1.0517389,0.9908537,0.9830700,0.9908537,0.9830700,0.0105273,0.0105273,5.1738907,5.1738907
,2,0.0200189,0.9819769,1.0582111,1.0549750,0.9969512,0.9822170,0.9939024,0.9826435,0.0105921,0.0211195,5.8211146,5.4975027
,3,0.0300284,0.9815917,1.0452667,1.0517389,0.9847561,0.9817793,0.9908537,0.9823554,0.0104626,0.0315820,4.5266668,5.1738907
,4,0.0400073,0.9813120,1.0419712,1.0493026,0.9816514,0.9814461,0.9885584,0.9821286,0.0103978,0.0419798,4.1971170,4.9302561
,5,0.0500168,0.9810742,1.0485028,1.0491425,0.9878049,0.9811950,0.9884076,0.9819418,0.0104949,0.0524747,4.8502787,4.9142508
,6,0.1000031,0.9802381,1.0523751,1.0507583,0.9914530,0.9806237,0.9899298,0.9812829,0.0526043,0.1050790,5.2375067,5.0758295
,7,0.1500198,0.9795893,1.0484949,1.0500037,0.9877974,0.9799023,0.9892189,0.9808226,0.0524423,0.1575214,4.8494889,5.0003673
,8,0.2000061,0.9790074,1.0484870,1.0496246,0.9877900,0.9792890,0.9888618,0.9804394,0.0524100,0.2099313,4.8486982,4.9624616
,9,0.3000092,0.9778528,1.0507583,1.0500025,0.9899298,0.9784389,0.9892178,0.9797726,0.1050790,0.3150104,5.0758295,5.0002509
,10,0.4000122,0.9765554,1.0481670,1.0495436,0.9874886,0.9772147,0.9887855,0.9791331,0.1048199,0.4198303,4.8167029,4.9543639



<bound method ModelBase.coef_norm of >


## Generating and storing Hyperparameters of each model into JSON

In [96]:
model_set=aml_leaderboard_df['model_id']
type(model_set)

pandas.core.series.Series

In [97]:
jsonDicts = []
for m in model_set.iteritems():
    m,model_name = m
    mod_best = h2o.get_model(model_name)
    jsonDicts.append(mod_best.params)

In [98]:
print(jsonDicts)

[{'model_id': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'StackedEnsemble_BestOfFamily_AutoML_20190415_235801', 'type': 'Key<Model>', 'URL': '/3/Models/StackedEnsemble_BestOfFamily_AutoML_20190415_235801'}}, 'training_frame': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'automl_training_py_13_sid_870a', 'type': 'Key<Frame>', 'URL': '/3/Frames/automl_training_py_13_sid_870a'}}, 'response_column': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ColSpecifierV3', 'schema_type': 'VecSpecifier'}, 'column_name': 'ACTION', 'is_member_of_frames': None}}, 'validation_frame': {'default': None, 'actual': None}, 'blending_frame': {'default': None, 'actual': None}, 'base_models': {'default': [], 'actual': [{'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': '

In [99]:
# def hyperparameter_json(jsonDicts,name):
#     i = json.dumps(jsonDicts,indent = 4)
#     f = open(run_id + name,'w')
#     print(i,file = f)
#     f.close()
n=run_id+'EmpAccess_500_hy_parameter.json'
dict_to_json(jsonDicts,n)

### Generating metadata and storing in JSON

In [100]:
meta_data['end_time'] = time.time()

In [101]:
meta_data

{'target': None,
 'classification': True,
 'run_time': 500,
 'run_id': 'y2l466LD5',
 'max_models': None,
 'start_time': 1555387074.0489674,
 'server_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project',
 'data_path': 'C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv',
 'test_path': None,
 'scale': False,
 'model_path': None,
 'balance': False,
 'balance_threshold': 0.2,
 'project': None,
 'end_time': 1555387614.5127614,
 'run_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project\\y2l466LD5',
 'nthreads': 1,
 'min_mem_size': 2,
 'analysis': 0,
 'Shape_Rows*Cols': (32769, 10),
 'X': ['RESOURCE',
  'MGR_ID',
  'ROLE_ROLLUP_1',
  'ROLE_ROLLUP_2',
  'ROLE_DEPTNAME',
  'ROLE_TITLE',
  'ROLE_FAMILY_DESC',
  'ROLE_FAMILY',
  'ROLE_CODE'],
 'levels': [['0', '1']],
 'variables': {'ACTION': 'enum',
  'RESOURCE': 'int',
  'MGR_ID': 'int',
  'ROLE_ROLLUP_1': 'int',
  'ROLE_ROLLUP_2': 'int',
  'ROLE_DEPTNAME': 'int',
  'ROLE_TITLE': 'int'

In [102]:
n=run_id+'EmpAccess_500_meta_data.json'
dict_to_json(meta_data,n)

#### Shutting down H2O

In [103]:
# Clean up
os.chdir(server_path)

In [104]:
h2o.cluster().shutdown()

H2O session _sid_870a closed.


#### Link of data set

https://www.kaggle.com/c/amazon-employee-access-challenge/overview

# Conclusion

# Contribution

# Citation

https://github.com/prabhuSub/Hyperparamter-Samples

https://github.com/nikbearbrown/CSYE_7245/tree/master/H2O



# License

Copyright (c) 2019 Urja Jain, Prakruthi Bagur Suryanarayanaprasad

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.